# ElasticNet Regresyon Modeli

### Gerekli Kütüphaneler

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.linear_model import RidgeCV , LassoCV , ElasticNetCV
from sklearn.model_selection import cross_val_score

#### Veri Seti

In [4]:
df = pd.read_csv("Hitters.csv")

df = df.dropna()

dms = pd.get_dummies(df[["League" , "Division" , "NewLeague" ] ])

y = df["Salary"]
X_ = df.drop(["Salary" , "League" , "Division" , "NewLeague"] , axis=1).astype("float64")

#bağımsız değişkenler
X = pd.concat([X_ , dms[["League_N" , "Division_W" , "NewLeague_N"]]], axis=1)

X_train , X_test , y_train , y_test = train_test_split(X,
                                                       y,
                                                       test_size=0.25,
                                                       random_state=42)

In [5]:
df.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N
5,594,169,4,74,51,35,11,4408,1133,19,501,336,194,A,W,282,421,25,750.0,A


In [6]:
df.shape

(263, 20)

In [7]:
enet_model = ElasticNet().fit(X_train, y_train)

In [8]:
enet_model.coef_

array([ -1.86256172,   8.70489065,   5.10426375,  -2.89875799,
        -1.28642985,   5.24343682,   6.04480276,  -0.14701495,
        -0.21566628,  -0.7897201 ,   1.80813117,   0.80914508,
        -0.61262382,   0.26816203,   0.27172387,  -0.36530729,
        19.2186222 , -31.16586592,   8.98369938])

In [9]:
enet_model.intercept_

-6.465955602111762

# Tahmin


In [10]:
enet_model.predict(X_train)[0:10]

array([325.74706292, 776.06632333, 522.86508419, 107.64091955,
       449.03139566, 997.76095723,  99.78828622, 311.33763086,
       418.50335021, 879.9502608 ])

In [11]:
enet_model.predict(X_test)[0:10]

array([ 577.79111731,  617.33202224, 1031.39113156,  364.95861575,
        489.51894393,  300.74185842,  604.522666  ,  465.34678732,
        901.44473965,  703.20357123])

In [12]:
y_pred = enet_model.predict(X_test)

In [14]:
#test hatası
np.sqrt(mean_squared_error(y_test, y_pred))

357.16765481812445

In [15]:
r2_score(y_test, y_pred)

0.4107022246932692

### Model Tuning

In [26]:

alphas = 10**np.linspace(10,-2,100)*0.5


In [27]:
enet_cv_model = ElasticNetCV(cv = 10 , max_iter=100000).fit(X_train , y_train)

In [17]:
enet_cv_model.alpha_

5230.7647364798695

In [18]:
enet_cv_model.intercept_

-38.51940558394301

In [19]:
enet_cv_model.coef_

array([ 0.62845434,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , -0.        ,  0.        ,  0.09788752,  0.        ,
        0.27265769,  0.19270075,  0.00758665,  0.3106529 ,  0.        ,
       -0.        ,  0.        , -0.        ,  0.        ])

In [20]:
#final modeli

In [22]:
enet_tuned = ElasticNet(alpha = enet_cv_model.alpha_).fit(X_train , y_train)

In [23]:
y_pred = enet_tuned.predict(X_test)

In [33]:
np.sqrt(mean_squared_error(y_test, y_pred))

394.15280563218795

In [25]:
?ElasticNet

In [28]:
enet_cv_model2 = ElasticNetCV(alphas = alphas, cv = 10 , max_iter=100000).fit(X_train , y_train)

In [29]:
enet_cv_model2.alpha_

5748.784976988678

In [30]:
enet_cv_model2.intercept_

-31.463121225640975

In [31]:
enet_cv_model2.coef_

array([ 0.61111381,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , -0.        ,  0.        ,  0.11212622,  0.        ,
        0.25252702,  0.18656722,  0.00444355,  0.30988823,  0.        ,
       -0.        ,  0.        , -0.        ,  0.        ])

In [38]:
enet_tuned2 = ElasticNet(alpha = enet_cv_model2.alpha_).fit(X_train , y_train)

In [39]:
y_pred2 = enet_tuned2.predict(X_test)

In [40]:
np.sqrt(mean_squared_error(y_test, y_pred2))

393.9753065850553